In [ ]:
  %cd /content/drive/MyDrive/Colab Notebooks/Deep Learning/New

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

- LDAcis 기반 Action 넘버가 매칭 된 모든 클러스터 데이터 로드

> 실습용 데이터
> - 0번째 Actor -> 7개의 Action
> - 1번째 Actor -> 4개의 Action

In [ ]:
cluster = 2
df_list = []
for i in range(cluster):
  with open(f'./data/cluster_df_{i}_action.pkl', 'rb') as f:
    temp_df = pickle.load(f)
  df_list.append(temp_df)

In [ ]:
df_list

In [ ]:
# 모든 클러스터에 대한 df를 병합!, index 새로부여
df = pd.concat(df_list, ignore_index=True)

In [ ]:
pd.set_option('display.max_colwidth', None)  # 긴 문자열 출력
pd.set_option('display.max_rows', None)  # 모든 행 출력

# 필터링된 데이터프레임 출력
df[(df["cluster"] == 1) & (df["action_cluster"] == 1) & df["review_clean"].str.contains("곰팡이", case=False, na=False)]


# 1. 만족도 계산 (satisfaction)
- 1.1 감성사전 불러오기
- 1.2 리뷰들의 각 토큰과 일치하는 감성 점수 구하는 함수 정의
- 1.3 감성 점수 구하기
- 1.4 Actor의 각 Action 별 감성점수 계산 후 정규화 -> 만족도

### 1.1 감성사전 불러오기
- KNU 한국어 감성사전
- http://dilab.kunsan.ac.kr/knusl.html

In [ ]:
import json  # JSON 파일을 읽고 쓰기 위한 도구

In [ ]:
with open('./data/SentiWord_info.json', encoding='utf-8-sig', mode='r') as f:
  sent_dict = json.load(f)

In [ ]:
print(sent_dict)
# {'word': 원래 단어 , 'word_root': 단어의 표준형태, 'polarity': 감성극성(ex.1 은 긍정, -1은 부정)}

In [ ]:
len(sent_dict)

### 1.2 리뷰들의 각 토큰과 일치하는 감성 점수 구하는 함수 정의


In [ ]:
len(df)

In [ ]:
# Test -> 각 리뷰 별로 단어 리스트(df["tagged_review"]) 중에 감성 사전에 매칭되는 단어(토큰)가 있는 경우 polarity  부여
result_list = []

for token in df["tagged_review"].iloc[0]:
  for sent_info in sent_dict:
    if token == sent_info["word"]: # 감성사전에 해당 token이 매칭되는가??
      result_list.append((sent_info["polarity"],sent_info["word"]))
print(result_list)


In [ ]:
# 사용자 함수 정의
def sentiment_score(sent_dict, review_token_list): # 감성사전, 리뷰의 토큰 리스트
  result_list = [] #(감점점수, 단어)를 담아둘 리스트

  for token in review_token_list:
    for sent_info in sent_dict:
      if token == sent_info["word"]: # 감성사전에 해당 token이 매칭되는가??
        result_list.append((sent_info["polarity"],sent_info["word"]))
  return result_list

In [ ]:
# 함수 테스트
sentiment_score(sent_dict, df["tagged_review"].iloc[0])

In [ ]:
! pip install konlpy

In [ ]:
from konlpy.tag import Okt

In [ ]:
okt = Okt()

In [ ]:
okt.tagset

# Adjective(형용사) : 명사의 성질(또는 상태)를 나타내는 품사 (ex. 좋다, 아름답다 등)
# Verb(동사) : 행위나 상태를 나타내는 품사 (ex 가다, 먹다 ,보다 ,하다 등)
# Adverb(부사) : 동사, 형용사, 전체 문장을 수식하는 품사 (ex. 천천히, 빨리, 매우 등)
# Exclamation(감탄사) : 감탄, 놀라움, 분노, 기쁨 등 감정을 독립적으로 표현하는 품사(ex. 와!, 아!, 우와! 등)

In [ ]:
# 품사 기준으로 토큰화 하는 함수
def okt_pos_taggind(text):
  # 품사태깅 + 어간 추출 + 정규화
  pos_words = okt.pos(text, stem = True, norm = True)
  tagged_list = []  # 감정 점수 계산에 활용될 단어들을 담아둘 리스트
  # 품사가 형용사, 동사, 부사, 감탄사 중에 하나라면 -> word를 추출
  for word, pos in pos_words:
    if pos in ["Adjective", "Verb", "Adverb", "Exclamation"]:
      tagged_list.append(word)
  return tagged_list

- 감성 점수 계산

In [ ]:
from tqdm.auto import tqdm

In [ ]:
sentiment = []

for review in tqdm(df["review"]):
  pos_tagged = okt_pos_taggind(review) # 감정에 특화 된 품사 기준으로 재토큰화
  score = sentiment_score(sent_dict, pos_tagged) # 추출된 단어리스트를 기반으로 감정 점수 계산
  sentiment.append(score)

In [ ]:
len(sentiment)

- 각 리뷰에 대한 평균 감정점수 구하기

In [ ]:
import numpy as np

In [ ]:
# 각 리뷰의 감성 점수(평균값)를 저장할 리스트
avg_sentiment_scores = []
for sentiment_result in sentiment:
   # 현재 리뷰의 모든 감성 점수를 정수형으로 형변환
   scores = [ int(result[0]) for result in sentiment_result]

   # 점수가  있으면 평균을 계산하고, 없으면 0을 저장
   avg_score = np.mean(scores) if scores  else 0
   avg_sentiment_scores.append(avg_score)

In [ ]:
print(len(avg_sentiment_scores))
print(avg_sentiment_scores[:5])

- df에 평균 감정 점수 추가

In [ ]:
df["sentiment_score"] = avg_sentiment_scores

In [ ]:
df.head(3)

## 1.4 각 Actor에 대한 Action 별 감정 점수 계산 후 정규화 -> 만족도

In [ ]:
action_setiments={}
for actor in df["cluster"].unique():
  actor_df = df[df["cluster"] == actor]

  for action in actor_df["action_cluster"].unique():  # 각 action 별 순회

    # 현재 액션에 해당되는 리뷰들의 감정 점수값을 지정
    action_score = actor_df[actor_df["action_cluster"] == action]["sentiment_score"]

    #평균 계산
    action_score_mean = np.mean(action_score)

    # 계산된 평균 감정 점수와 각 액터 및 액션을 딕셔너리에 저장
    action_setiments[f"Actor{actor}_Action{action}"] = action_score_mean

In [ ]:
# 키값만 저장 -> 기회영역 시각화 할 때 사용
actions = action_setiments.keys()

In [ ]:
action_setiments

- 감정점수 정규화(스케일링) 수행

In [ ]:
# action_setiments['Actor1_Action1'] = 0.1050922073252171
# action_setiments['Actor1_Action3'] = -0.100424483956672

In [ ]:
action_setiments

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# 1. 정규화 할 데이터(액션별 평균감정점수) 가져오기
raw_scores = list(action_setiments.values())
# 2. 형변환 -> 2차원 배열로 형변환 (MinMaxScaler가 요구하는 차원)
scores_array = np.array(raw_scores).reshape(-1 ,1)
# 3. 스케일러 초기화
scaler = min_max_scaler = MinMaxScaler(feature_range= (0,10)) # 끝값 포함!!!
# 4. 스케일링 수행
scaled_scores = scaler.fit_transform(scores_array)

# 5. 스케일링 결과를 1차원 리스트로 형변환(소수점 4자리까지 반올림)
scaled_scores_list = scaled_scores.flatten().tolist()
scaled_scores_list_round = [round(score, 4) for score in scaled_scores_list]

# 6. 스케일링 된 점수를 기존 딕셔너리(action_sentiments)에 업데이트
for key,new_score in zip(action_setiments.keys(), scaled_scores_list_round):
  action_setiments[key] = new_score

# 7. df화
action_summary_df = pd.DataFrame(action_setiments.items(),columns = ["Action","satisfaction"])

# 2. 중요도(Importance)
- importance = (빈도수/전체 빈도수) *100
- 2.1 중요도 점수 구하기 -> 전체 토픽에 대한 비중
- 2.2 중요도 점수에 대한 정규화(스케일링)

### 2.1 중요도 점수 구하기

In [ ]:
# 모든 리뷰에 대해서 "Actor0_Action0" 형태의 문자열을 생성

actor_action_labels = []
for actor, action in zip(df["cluster"], df["action_cluster"]):
  actor_action_labels.append(f"Actor{actor}_Action{action}")

In [ ]:
# 빈도수 계산
from collections import Counter

In [ ]:
label_frequency = Counter(actor_action_labels)
label_frequency

In [ ]:
# label_frequency['Actor1_Action1'] = 553

In [ ]:
label_frequency

In [ ]:
# 전체 빈도수 계산
total_count = sum(label_frequency.values())
total_count

In [ ]:
# 중요도 계산
# 중요도 = (해당 action의 빈도수 / 전체 빈도수) * 100
importance_dict = { label: (count/total_count)*100 for label, count in label_frequency.items()}
importance_dict

In [ ]:
# 1. 정규화 할 데이터(액션별 정규화 이전의 중요도) 가져오기

raw_scores = list(importance_dict.values())

# 2. 형변환 -> 2차원 배열로 형변환(MinMaxScaler가 요구하는 차원)

scores_array = np.array(raw_scores).reshape(-1, 1)

# 3. 스케일러 초기화

scaler = MinMaxScaler(feature_range = (0, 10)) # 끝값 포함!

# 4. 스케일링 수행

scaled_scores = scaler.fit_transform(scores_array)

# 5. 스케일링 결과를 1차원 리스트로 형변환(소수점 4자리까지 반올림)

scaled_scores_list = scaled_scores.flatten().tolist()

scaled_scores_list_round = [round(score, 4) for score in scaled_scores_list]


# 6. 스케일링 된 점수를 기존 딕셔너리(importance_dict)에 업데이트

for key, new_score in zip(importance_dict.keys(), scaled_scores_list_round) :
    importance_dict[key] = new_score
importance_dict

In [ ]:
# 7. df에 추가
action_summary_df["importance"] = list(importance_dict.values())
action_summary_df

# 3. 기회 점수(Opportunity)
- 3.1 기회점수 계산
- 3.2 기회 영역 시각화
- 만약에 고객이 어떤 행동을 중요하게 여기지만, 그에 비해 만족도가 낮다면 -> 그 차이가 클수록 고객에게 개선의 여지가 많다(가치가 높다)는 걸 의미하는 분석 방법

In [ ]:
# 기회 점수 계산 함수 정의
def opportunity_score(satisfaction, importance):
  return importance + max(satisfaction - importance,0)

In [ ]:
# 계산
opportunity_list=[]
for s, i in zip(action_summary_df["satisfaction"], action_summary_df["importance"]):
  opportunity_list.append(opportunity_score(s,i))
opportunity_list

In [ ]:
# df에 추가
action_summary_df["opportunity_score"] = opportunity_list
action_summary_df

In [ ]:
# 저장
action_summary_df.to_csv("./data/sent_impo_oppo_score.csv", index = False)

- 기회영역 시각화

In [ ]:
# 만족도, 중요도, 액션이름(라벨) 가져옴
satisfaction = action_summary_df["satisfaction"]
importance = action_summary_df["importance"]
actions = action_summary_df["Action"]
actions # actor0 -> 7 개, actor1 -> 4개

In [ ]:
# 각 액션마다 무작위 색상을 생성
colors = np.random.rand(len(actions),3)
colors

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize = (17, 10))

# 산점도
plt.scatter(
    importance,
    satisfaction,
    s = 35,          # 마커 크기
    c = colors,      # 각 마커의 색상(임의의 랜덤값)
    edgecolors = 'black' # 마커의 테두리

)

plt.xlabel('Importance')     # x축 이름
plt.ylabel('Satisfaction')   # y축 이름


# 첫번째 영역 분할 선
xdata = [0, 10]
ydata = [satisfaction.mean(), 10]
plt.plot(xdata, ydata, 'r')

# 두 번째 영역 분할 선
x_data = [importance.mean(), 10]
y_data = [0, 10]
plt.plot(x_data, y_data, 'g')

# 각 액션에 이름(텍스트) 추가
for i, action in enumerate(actions):
    plt.text(
        importance[i], satisfaction[i],
        action,
        fontsize = 10,
        ha = 'left' # 왼쪽 정렬
    )

# 범례 표시를 위한 빈 산점도 그리기
for i, action in enumerate(actions):
    plt.scatter(
        [], [],
        s = 10,
        c = [colors[i]],
        label = action,
        edgecolors = 'black'
    )

# 범례 표시
plt.legend(
    title = 'Actions',
    fontsize = 8,
    title_fontsize = '10',
    loc = 'upper left',   # 범례들이 왼쪽 상단에 위치
    bbox_to_anchor=(1, 1) # 범례를 플롯 외부 오른쪽 상단에 고정
)

plt.grid()

# 그래프를 저장한다
plt.savefig(
    './data/Opportunity_area.png',
    dpi = 300,
    bbox_inches = 'tight'  # 여백을 최소화
)
plt.show()
